# Deep Learning (TensorFlow, Keras): Image Multi-class Classifier (Part 1)
In this project, a model is trained to perform multi-class classification for apple, banana and orange pictures. None pretrained model is used, the convolutiona neuronal network will be design from zero. This document is the first part of the whole training process.

The dataset can be found in (805 MB):

https://www.kaggle.com/datasets/shivamardeshna/fruits-dataset

## Iteration 1: CNN creation and training (learning_rate=1e-4) without data augmentation

In [ ]:
# (height, width, channels)
input_shape = (224, 224, 3)
batch_size = 12
learning_rate = 1e-4
path_dataset = '../../fruits_dataset'
folder_apple = 'apple'
folder_banana = 'banana'
folder_orange = 'orange'
folder_models = '../../models'

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, Flatten, Dense, AvgPool2D,MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [ ]:
# Find how many apple, banana, and orange images exist
apple_imgs = os.listdir(os.path.join(path_dataset,folder_apple))
banana_imgs = os.listdir(os.path.join(path_dataset,folder_banana))
orange_imgs = os.listdir(os.path.join(path_dataset,folder_orange))
print(f'Apple images found: {len(apple_imgs)}')
print(f'Banana images found: {len(banana_imgs)}')
print(f'Orange images found: {len(orange_imgs)}')

Classes are balanced.

### No Data augmentation

In [ ]:
def load_data(path, input_shape=input_shape, batch_size=batch_size, seed=123, validation_split=0.2):
    """Function to create 2 ImageDataGenerators to split dataset into train and validation datasets.
    Data augmentation is not implemented for the validation dataset."""
    height, width = input_shape[:2]
    datagen = ImageDataGenerator(rescale=1.0/255, zoom_range=0,
        horizontal_flip=True, vertical_flip=False,
        height_shift_range=0, width_shift_range=0,
        brightness_range=(0.99, 1.0), rotation_range=0,
        validation_split=validation_split
    )
    train_data = datagen.flow_from_directory(path,
        target_size=(height, width), batch_size=batch_size,
        class_mode='sparse', subset='training', seed=seed
    )
    val_datagen = ImageDataGenerator(rescale=1.0/255,
        validation_split=validation_split
    )
    val_data = val_datagen.flow_from_directory(path,
        target_size=(height, width), batch_size=batch_size,
        class_mode='sparse', subset='validation', seed=seed
    )
    return train_data, val_data

# Split training and validation datasets
train, val = load_data(path_dataset)

print(f"Classes found: {train.class_indices}")
print(f"Training images: {train.samples}")
print(f"Validation images: {val.samples}")

In [ ]:
# Obtain images and target
images, labels = next(train)

# Show 8 training images (batch_size=8)
figure, axes = plt.subplots(nrows=2,ncols=4, figsize=(8, 6))
for item in zip(axes.ravel(), images, labels):
    axes, image, target = item
    axes.imshow(image)
    axes.set_title(f'Target: {target:0.0f}')
    axes.set_xticks([])
    axes.set_yticks([])
plt.tight_layout()
plt.show()

# Image dimensions
print(images.shape)

### Model training

In [ ]:
def create_model(input_shape=input_shape, learning_rate=learning_rate):
    """Function to create a CNN model from scratch."""
    model = Sequential()
    model.add(Conv2D(6, (5, 5), padding='same', activation='relu', input_shape=input_shape))
    model.add(AvgPool2D(pool_size=(2, 2)))
    model.add(Conv2D(16, (5, 5), padding='valid', activation='relu'))
    model.add(AvgPool2D(pool_size=(2, 2)))
    model.add(Conv2D(16, (5, 5), padding='valid', activation='relu'))
    model.add(AvgPool2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(3, activation='softmax'))           # 3 classes
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def train_model(model, train_data, val_data, epochs, version_model):
    """Function to train the model and save the best one
    according to the validation accuracy."""
    file_name = os.path.join(folder_models,f'sparse_model_v{version_model}.h5')
    
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=0),
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6, verbose=0),
        ModelCheckpoint(file_name, monitor='val_accuracy', save_best_only=True, verbose=1)
    ]

    history = model.fit(train_data, validation_data=val_data,
              epochs=epochs, callbacks=callbacks, verbose=2)

    return model, history

In [ ]:
epochs = 30
version_model = 1
print(f"Parameters: batch_size = {batch_size}, learning_rate = {learning_rate}, epochs = {epochs}")

In [ ]:
# Create and train the model v1
model = create_model()
model.summary()

In [ ]:
print(f"TensorFlow Version: {tf.__version__}")

# Ensure GPU is available
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0],True)
    print("GPU is available and memory growth is enabled.")
else:
    print("GPU not available, training will be on CPU.")

In [ ]:
# Train the model
model, history_stage1 = train_model(model, train, val, epochs=epochs, version_model=version_model)

**Result 1:** val_accuracy=%.

In [ ]:
pd.DataFrame(history_stage1.history).plot(figsize=(12, 4))
plt.show()

In [ ]:
# Save model
# model.save(os.path.join(folder_models,f'binary_model_v{version_model}.keras'))

In the next iteration, the model will be retrained, data augmentation and fine-tuning (last 10 layers) will be performed.